In [49]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [9]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data2 = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

scaler = StandardScaler()


In [12]:
#Create a DataFrame with the scaled data
df_norm = pd.DataFrame(scaler.fit_transform(df_market_data2), columns=df_market_data2.columns, index=df_market_data2.index)


#Display sample data
df_norm.head(5)


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [110]:
# Create a list with the number of k-values from 1 to 11
k= list (range(1,11))

In [111]:
# Create an empty list to store the inertia values
inertia= []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
    k_model = KMeans (n_clusters =i, random_state = 0)
    k_model.fit (df_norm)
    inertia.append (k_model.inertia_)
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list


/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to

In [136]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k":k, "Inertia": inertia}
df_elbow1 = pd.DataFrame (elbow_data)
# Create a DataFrame with the data to plot the Elbow curve


In [137]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# Plot the DataFrame to identify the optimal value for k
elbow1=df_elbow1.hvplot.line(
    x="k", 
    y="Inertia", 
    title="Elbow Curve", 
    xticks=k)

elbow1


:Curve   [k]   (Inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  The value is 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [27]:
# Initialize the K-Means model using the best value for k
model= KMeans (n_clusters=4, random_state =0)

In [28]:
# Fit the K-Means model using the scaled data
model.fit (df_norm)

/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=4, random_state=0)

In [30]:
# Predict the clusters to group the cryptocurrencies using the scaled data
prediction_df = model.predict (df_norm)

# Print the resulting array of cluster values.
prediction_df

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [31]:
# Create a copy of the DataFrame
market_prediction_df= df_norm.copy()

In [35]:
# Add a new column to the DataFrame with the predicted clusters
market_prediction_df ['kmeans_segments'] = prediction_df

# Display sample data
market_prediction_df.head(3)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,kmeans-segments,kmeans_segments
coin_id,,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0,0


In [145]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
prediction1= market_prediction_df.hvplot.scatter(
    x='price_change_percentage_24h',
    y= 'price_change_percentage_7d',
    by ='kmeans_segments',
    colormap ='Tab10',
    size=20,
    alpha=.5,
    hover_cols=['coin_id']
)
prediction1

:NdOverlay   [kmeans_segments]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [43]:
# Create a PCA model instance and set `n_components=3`.
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [45]:
# Use the PCA model with `fit_transform` to reduce to 
info_pca=pca.fit_transform (market_prediction_df)
# three principal components.

# View the first five rows of the DataFrame. 
info_pca[:5]

array([[ 1.76708203, -0.84095177, -0.53061562],
       [ 1.65512889, -0.62285242, -1.10004707],
       [-1.04605685, -0.33893146,  0.68460831],
       [-1.04870151, -0.36721571,  0.52644123],
       [ 2.82089608, -1.68886608, -0.9535845 ]])

In [46]:
# Retrieve the explained variance to determine how much information 
pca.explained_variance_ratio_



array([0.41681554, 0.28976446, 0.18396795])

In [52]:
evra= np.array ([0.41681554, 0.28976446, 0.18396795])
total_ev=np.sum (evra)
total_ev

0.89054795

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.89054795

In [72]:
# Create a new DataFrame with the PCA data.
df_info_pca=pd. DataFrame(
    info_pca,
    columns=['PCA1','PCA2','PCA3']
)
# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
df_info_pca ['coin_id']= df_norm.index

# Set the coinid column as index

df_info_pca = df_info_pca[['coin_id', 'PCA1', 'PCA2', 'PCA3']]
df_info_pca.set_index('coin_id', inplace=True)

# Display sample data
df_info_pca.head (3)

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,1.767082,-0.840952,-0.530616
ethereum,1.655129,-0.622852,-1.100047
tether,-1.046057,-0.338931,0.684608


---

### Find the Best Value for k Using the PCA Data

In [73]:
# Create a list with the number of k-values from 1 to 11
k_pca=list (range(1,11))

In [75]:
# Create an empty list to store the inertia values
inertia_pca=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    k_model=KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_info_pca)
    inertia_pca.append (k_model.inertia_)

/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to

In [134]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "Inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
# Create a DataFrame with the data to plot the Elbow curve
df_elbow.head(3)

,k,Inertia
0,1,287.000000
1,2,198.571818
2,3,123.190482


In [138]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow2=df_elbow.hvplot.line(
    x="k", 
    y="Inertia", 
    title="Elbow Curve PCA", 
    xticks=k
)
elbow2

:Curve   [k]   (Inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it's the same.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [86]:
# Initialize the K-Means model using the best value for k
model= KMeans (n_clusters=4, random_state =0)

In [87]:
# Fit the K-Means model using the PCA data
model.fit (df_info_pca)

/Users/joel/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=4, random_state=0)

In [89]:
# Predict the clusters to group the cryptocurrencies using the scaled data
prediction = model.predict (df_info_pca)

# Print the resulting array of cluster values.
prediction


array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [99]:
# Create a copy of the DataFrame with the PCA data
df_prediction_pca = df_info_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_prediction_pca['kmean_segments']=prediction

# Display sample data
df_prediction_pca.head(5)

,PCA1,PCA2,PCA3,kmean_segments
coin_id,,,,
bitcoin,1.767082,-0.840952,-0.530616,2
ethereum,1.655129,-0.622852,-1.100047,2
tether,-1.046057,-0.338931,0.684608,0
ripple,-1.048702,-0.367216,0.526441,0
bitcoin-cash,2.820896,-1.688866,-0.953584,2


In [143]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
prediction2=df_prediction_pca.hvplot.scatter(
    x='PCA1',
    y= 'PCA2',
    by ='kmean_segments',
    colormap ='Tab10',
    size=20,
    alpha=.5,
    hover_cols=['coin_id']
)
prediction2

:NdOverlay   [kmean_segments]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [139]:
# Create the first Elbow plot
(elbow1 + elbow2).cols(1)



:Layout
   .Curve.I  :Curve   [k]   (Inertia)
   .Curve.II :Curve   [k]   (Inertia)

In [146]:
# Composite plot to contrast the clusters
(prediction1 + prediction2).cols(1)
# YOUR CODE HERE!

:Layout
   .NdOverlay.I  :NdOverlay   [kmeans_segments]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [kmean_segments]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**  Working with PCA data results in more concentrated clusters. It aslo results in more entries in o and 2 compared to the initial analysis.And we have more loosers working with PCA data.